In [370]:
import pandas as pd
import csv
import math as m
import gurobipy as GRB
from ipycanvas import Canvas

hor = open("hor.csv")
csv_hor = list(csv.reader(hor))

ver = open("ver.csv")
csv_ver = list(csv.reader(ver))

horshtraf = open("hor_shtraf.csv")
csv_horshtraf = list(csv.reader(horshtraf))

vershtraf = open("ver_shtraf.csv")
csv_vershtraf = list(csv.reader(vershtraf))

lo = GRB.Model("LO model")

In [371]:
x = lo.addVars(
    [
        (i,j)
        for i in range(10)
        for j in range(10)
    ],
    vtype=GRB.GRB.BINARY,
    name="x[i,j]",
)

y = lo.addVars(
    [
        (i,j)
        for i in range(10)
        for j in range(10)
    ],
    vtype=GRB.GRB.BINARY,
    name="y[i,j]",
)

a = lo.addVars(
    [
        (i,j)
        for i in range(10)
        for j in range(10)
    ],
    vtype=GRB.GRB.BINARY,
    name="a[i,j]",
)

b = lo.addVars(
    [
        (i,j)
        for i in range(10)
        for j in range(10)
    ],
    vtype=GRB.GRB.BINARY,
    name="b[i,j]",
)

Budget = int(input())

In [372]:
objective = GRB.quicksum(x[i,j]*(100/int(csv_hor[i][j])) for i in range(10) for j in range(9)) \
    - GRB.quicksum(a[i,j]*(3000/((int(csv_hor[i][j])+30)*int(csv_hor[i][j]))) for i in range(10) for j in range(9)) \
    + GRB.quicksum(y[i,j]*(100/int(csv_ver[i][j])) for i in range(9) for j in range(10)) \
    - GRB.quicksum(b[i,j]*(3000/((int(csv_ver[i][j])+30)*int(csv_ver[i][j]))) for i in range(9) for j in range(10))

In [373]:
lo.addConstr( x[0,0] + y[0,0] == 1 )
lo.addConstr( x[9,0] == y[8,0] )
lo.addConstr( x[0,8] == y[0,9] )
lo.addConstr( x[9,8] + y[8,9] == 1 )

lo.addConstrs( x[0,j] == x[0,j+1] + y[0,j+1] for j in range (8))
lo.addConstrs( y[i,0] == x[i+1,0] + y[i+1,0] for i in range (8))

lo.addConstrs( x[i+1,j+1] + y[i+1,j+1] == x[i+1,j] + y[i,j+1] for i in range(8) for j in range(8))

lo.addConstrs( x[9,j] + x[9,j+1] == y[8,j+1] for j in range (8))
lo.addConstrs( y[i,9] + x[i+1,8] == y[i+1,9] for i in range (8))

lo.addConstr( GRB.quicksum(a[i,j]*int(csv_horshtraf[i][j]) for i in range (10) for j in range (9)) + GRB.quicksum(b[i,j]*int(csv_vershtraf[i][j]) for i in range (9) for j in range (10)) <= Budget)
lo.addConstrs( a[i,j] <= x[i,j] for i in range (10) for j in range (9))
lo.addConstrs( b[i,j] <= y[i,j] for i in range (9) for j in range (10))

lo.update()

In [374]:
lo.ModelSense = GRB.GRB.MINIMIZE
lo.setObjective(objective)
lo.update()
lo.optimize()

solution_hor = lo.getAttr('X', x)
solution_ver = lo.getAttr('X', y)
solution_horshtraf = lo.getAttr('X', a)
solution_vershtraf = lo.getAttr('X', b)
sum = GRB.quicksum(x[i,j].x for i in range(10) for j in range(9))+GRB.quicksum(y[i,j].x for i in range(9) for j in range(10))
sumshtraf = GRB.quicksum(a[i,j].x for i in range(10) for j in range(9))+GRB.quicksum(b[i,j].x for i in range(9) for j in range(10))
print(sum, sumshtraf)

# for i in range (10):
#     for j in range (9):
#         if solution_hor[i,j] == 1 :
#             print( '(',i,',',j,'):', solution_hor[i,j],end='; ')
# print()
# for i in range (10):
#     for j in range (9):
#         if solution_horshtraf[i,j] == 1 :
#             print( '(',i,',',j,'):', solution_horshtraf[i,j],end='; ')
# print()           
# for i in range (9):
#     for j in range (10):
#         if solution_ver[i,j] == 1 :
#             print( '(',i,',',j,'):', solution_ver[i,j],end='; ')
# print()
# for i in range (9):
#     for j in range (10):
#         if solution_vershtraf[i,j] == 1 :
#             print( '(',i,',',j,'):', solution_vershtraf[i,j],end='; ')

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 281 rows, 400 columns and 900 nonzeros
Model fingerprint: 0xf008abc5
Variable types: 0 continuous, 400 integer (400 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e-01, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+02]
Presolve removed 36 rows and 81 columns
Presolve time: 0.01s
Presolved: 245 rows, 319 columns, 848 nonzeros
Variable types: 0 continuous, 319 integer (319 binary)
Found heuristic solution: objective 17.1322844

Root relaxation: objective 1.658256e+01, 113 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0  

In [376]:
canvas = Canvas(width=600, height=600)
canvas.line_width = 1
canvas.stroke_style = "black"
for i in range (10):
    canvas.fill_text(str(i),i*50+50,550)
    canvas.fill_text(str(i),0,500-i*50)
    canvas.begin_path()
    canvas.move_to(i*50+50,550)
    canvas.line_to(i*50+50,50)
    canvas.stroke()
    canvas
    canvas.begin_path()
    canvas.move_to(0,500-i*50)
    canvas.line_to(500,500-i*50)
    canvas.stroke()
    canvas

q=50
w=500
i=0
j=0
t=0
canvas.begin_path()
canvas.move_to(q,w)
canvas.line_width = 10
canvas.line_cap = "square"

while t == 0:
    if solution_hor[i,j] == 1:
        canvas.line_to(q+50,w)
        canvas.stroke()
        j=j+1
        q=q+50
    else:
        canvas.line_to(q,w-50)
        canvas.stroke()
        i=i+1
        w=w-50
    if i+j == 18:
        t = 1

canvas.stroke_style = "red"
for i in range (10):
    for j in range (10):
        if solution_horshtraf[i,j] == 1:
            canvas.begin_path()
            canvas.move_to(50*(j+1),500-50*i)
            canvas.line_to(50*(j+2),500-50*i)
            canvas.stroke()
            canvas
        if solution_vershtraf[i,j] == 1:
            canvas.begin_path()
            canvas.move_to(50*(j+1),450-50*i)
            canvas.line_to(50*(j+1),500-50*i)
            canvas.stroke()
            canvas

canvas

Canvas(height=600, width=600)